In [1]:
import pickle
import numpy as np
import pandas as pd
import centrex_TlF as centrex
import matplotlib.pyplot as plt
from uncertainties import ufloat
from dataclasses import dataclass
import scipy.constants as cst
from pathlib import Path

In [2]:
@dataclass
class PeakMeasurement:
    height: ufloat
    width: ufloat

In [3]:
def thermal_population(J, T, B=6.66733e9, n = 100):
    """calculate the thermal population of a given J sublevel
    Args:
        J (int): rotational level
        T (float): temperature [Kelvin]
        B (float, optional): rotational constant. Defaults to 6.66733e9.
        n (int, optional): number of rotational levels to normalize with. 
                            Defaults to 100.
    Returns:
        float: relative population in a rotational sublevel
    """
    c = 2*np.pi*cst.hbar*B/(cst.k*T)
    g = lambda J: 4*(2*J+1)
    a = lambda J: -c*J*(J+1)
    Z = np.sum([g(i)*np.exp(a(i)) for i in range(n)])
    return g(J)*np.exp(a(J))/Z

def J_levels(J):
    """calculate the number of hyperfine sublevels per J rotational level
    Args:
        J (int): rotational level
    Returns:
        int: number of levels
    """
    return 4*(2*J + 1)

In [4]:
def calculate_photons_sim(transition, df, T):
    J = int(transition[2])
    idx = np.where(df.transition == transition)[0][0]
    signal = df.signal[idx]
    ρ_thermal = thermal_population(J, T)
    indices = np.array(df.index)
    others = indices[indices != idx]
    # calculate photons in transition from other transitions
    # based on measured signals
    nphotons = []
    nphotons_sim = df["#photons_sim"].values
    levels = df.levels.values
    for idy in others:
        J_other = int(df.transition[idy][2])
        ρ_thermal_other = thermal_population(J_other, T)
        ratio_signal = signal/df.signal[idy]
        ratio_levels_sim = levels[idy]/J_levels(J_other)
        ratio_levels_tocalc = levels[idx]/J_levels(J)
        _ = ratio_signal * nphotons_sim[idy] * ratio_levels_sim * ρ_thermal_other / \
            ( ratio_levels_tocalc * ρ_thermal )
        nphotons.append(_)
    nphotons = np.array(nphotons)
    transitions = df.transition.values[others]
    return [*zip(transitions, nphotons)]

In [5]:
transitions = [
    centrex.transitions.LaserTransition("R1", F1=3/2, F=1),
    centrex.transitions.LaserTransition("R1", F1=3/2, F=2),
    centrex.transitions.LaserTransition("R1", F1=5/2, F=2),
    centrex.transitions.LaserTransition("R1", F1=5/2, F=3),
    centrex.transitions.LaserTransition("Q1", F1=1/2, F=0),
    centrex.transitions.LaserTransition("Q1", F1=1/2, F=1),
    centrex.transitions.LaserTransition("Q1", F1=3/2, F=1),
]

In [6]:
path = Path()

simulated_heights = {}
for p in path.glob("*.pkl"):
    print(p)
    with open(p, 'rb') as f:
        data = pickle.load(f)
        simulated_heights[str(p.stem)] = np.max(data['signal'])

Q(1) F1'=1_2 F'=0.pkl
Q(1) F1'=1_2 F'=1.pkl
Q(1) F1'=3_2 F'=1.pkl
R(1) F1'=3_2 F'=1.pkl
R(1) F1'=3_2 F'=2.pkl
R(1) F1'=5_2 F'=2.pkl
R(1) F1'=5_2 F'=3.pkl


In [7]:
# order the simulations in the same order as transitions
_ = []
for transition in transitions:
    _.append(simulated_heights[transition.get_string_os()])
simulated_heights = _
del _

In [8]:
measurements = [
    PeakMeasurement(ufloat(23.74, 0.76), ufloat(10.67, 0.55)),
    PeakMeasurement(ufloat(17.52, 0.47), ufloat(20.77, 1.02)),
    PeakMeasurement(ufloat(13.27, 0.54), ufloat(15.80, 1.11)),
    PeakMeasurement(ufloat(7.30, 0.40), ufloat(31.65, 3.52)),
    PeakMeasurement(ufloat(10.34, 0.52), ufloat(18.13, 1.61)),
    PeakMeasurement(ufloat(183.59, 2.98), ufloat(7.27, 0.18)),
    PeakMeasurement(ufloat(38.01, 0.96), ufloat(10.42, 0.42)),
]

In [9]:
# without vibrational, edit such that none exceed 100 photons cycled
branching = [
    0.7094209682686317,
    0.7118596758679805,
    0.5249999224639202,
    0.5235469186730339,
    0.99, #0.9999999938796454,
    0.99, #0.9998733406269951,
    0.8907056877229965
]

In [10]:
df = pd.DataFrame(
    {
        "levels": [t.n_ground for t in transitions],
        "signal sim": [round(s,1) for s in simulated_heights],
        "signal": [m.height for m in measurements],
    }
)
df.index = [t.get_string() for t in transitions]
df

,levels,signal sim,signal
R(1) F1'=3/2 F'=1,12,2.8,23.7+/-0.8
R(1) F1'=3/2 F'=2,11,2.3,17.5+/-0.5
R(1) F1'=5/2 F'=2,11,1.8,13.3+/-0.5
R(1) F1'=5/2 F'=3,5,0.9,7.3+/-0.4
Q(1) F1'=1/2 F'=0,6,1.0,10.3+/-0.5
Q(1) F1'=1/2 F'=1,12,14.5,183.6+/-3.0
Q(1) F1'=3/2 F'=1,12,3.6,38.0+/-1.0


In [11]:
df_normalized = df.copy()
df_normalized["signal sim"] = round(df_normalized["signal sim"]/df_normalized["signal sim"][0],2)
df_normalized["signal"] = df_normalized["signal"]/df_normalized["signal"][0]
df_normalized["ratio difference"] = np.abs((df_normalized["signal sim"] - df_normalized["signal"]) / df_normalized["signal sim"])
df_normalized

,levels,signal sim,signal,ratio difference
R(1) F1'=3/2 F'=1,12,1.00,1.0+/-0,0.0+/-0
R(1) F1'=3/2 F'=2,11,0.82,0.738+/-0.031,0.10+/-0.04
R(1) F1'=5/2 F'=2,11,0.64,0.559+/-0.029,0.13+/-0.05
R(1) F1'=5/2 F'=3,5,0.32,0.307+/-0.020,0.04+/-0.06
Q(1) F1'=1/2 F'=0,6,0.36,0.436+/-0.026,0.21+/-0.07
Q(1) F1'=1/2 F'=1,12,5.18,7.73+/-0.28,0.49+/-0.05
Q(1) F1'=3/2 F'=1,12,1.29,1.60+/-0.07,0.24+/-0.05


In [12]:
def style_within_percentage(v, props = "", **kwargs):
    percentage = kwargs.get("percentage", 0.1)
    if v["ratio difference"].n == 0:
        return ['color:white;background-color:blue;opacity: 50%;']*len(v)
    elif v["ratio difference"].n <= percentage:
        return [props]*len(v)
    else:
        return None

In [13]:
percentage_good = 0.15

cmd_string = "df_normalized.style"
for idx, transition in enumerate(transitions):
    transition = transition.get_string()
    value = df_normalized["ratio difference"][idx].n
    cmd_string += (
        f".apply("
        f"style_within_percentage,"
        f"props = 'color:white;background-color:green;opacity: 50%;',"
        f"percentage = {percentage_good},"
        "axis = 1"
        ')'
    )
df_styled = eval(cmd_string)
df_styled

,levels,signal sim,signal,ratio difference
R(1) F1'=3/2 F'=1,12,1.000000,1.0+/-0,0.0+/-0
R(1) F1'=3/2 F'=2,11,0.820000,0.738+/-0.031,0.10+/-0.04
R(1) F1'=5/2 F'=2,11,0.640000,0.559+/-0.029,0.13+/-0.05
R(1) F1'=5/2 F'=3,5,0.320000,0.307+/-0.020,0.04+/-0.06
Q(1) F1'=1/2 F'=0,6,0.360000,0.436+/-0.026,0.21+/-0.07
Q(1) F1'=1/2 F'=1,12,5.180000,7.73+/-0.28,0.49+/-0.05
Q(1) F1'=3/2 F'=1,12,1.290000,1.60+/-0.07,0.24+/-0.05
